# Stateful Applications with CrewAI

This notebook demonstrates building a **stateful culinary assistant** using CrewAI that finds recipes based on ingredients and dietary restrictions, then provides step-by-step cooking instructions.

## What You'll Learn

- How to create a CrewAI **Agent** with a specific role, goal, and backstory
- How to define **Tasks** that chain together using the `context` parameter (stateful handoff)
- How to assemble a **Crew** and execute it with automatic planning

## Key Concept: Statefulness via Task Context

CrewAI supports statefulness through **task context** — a downstream task can reference the output of an upstream task, allowing the agent to carry forward prior results without manual wiring.

## Step 1: Import Dependencies & Configure the LLM

We import CrewAI's core building blocks — `Agent`, `Task`, `Crew` — and the `LLM` wrapper that lets us specify which language model powers our agent.

In [11]:
import os
from crewai import Agent, Task, Crew, LLM

from dotenv import load_dotenv
load_dotenv()

True

In [12]:
# Use CrewAI's native OpenAI provider pointed at Groq's API.
# "provider" must be explicit so CrewAI doesn't strip the "openai/" prefix
# from the model name — Groq requires the full name "openai/gpt-oss-20b".
llm = LLM(
    model="openai/gpt-oss-20b",
    provider="openai",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0,
)

In [13]:
# llm = ChatGroq(temperature=0,
#              model_name="openai/gpt-oss-20b")
# llm.invoke("Hello, how are you?").content

## Step 2: Define User Inputs

These variables act as the "user request" — the ingredient they want to cook with and any dietary restrictions. Try changing these values to see how the agent adapts!

In [14]:
main_ingredient = "tomato"          # Primary ingredient the recipe must include
dietary_restrictions = "shrimps"    # Dietary constraint (e.g., allergens or preferences)

## Step 3: Create the Agent

A CrewAI **Agent** is defined by three key attributes:
- **`role`** — The persona the agent assumes (e.g., "Culinary Assistant")
- **`goal`** — What the agent is trying to achieve
- **`backstory`** — Context that shapes how the agent reasons and responds

Setting `verbose=True` lets us observe the agent's chain-of-thought during execution.

In [16]:
culinary_assistant = Agent(
    llm=llm,
    role="Culinary Assistant",
    backstory=(
        "An experienced culinary assistant skilled in finding and tailoring "
        "recipes based on ingredients and dietary needs, and providing clear, "
        "step-by-step cooking instructions"
    ),
    goal="Find recipes, filter them to meet dietary preferences, and guide user through recipe steps.",
    verbose=True,  # Prints the agent's reasoning steps
)

## Step 4: Define Tasks (with Stateful Context)

Each **Task** tells an agent *what* to do (`description`) and *what output to produce* (`expected_output`).

The `context` parameter on the second task is what makes this **stateful** — it passes the output of `find_and_filter_recipes` into `guide_recipe_steps`, so the agent builds on its previous work rather than starting from scratch.

In [17]:
# Task 1: Find and filter recipes based on user inputs
find_and_filter_recipes = Task(
    description=(
        f"Find recipes that use the ingredient: {main_ingredient} and "
        f"filter them to meet dietary restrictions: {dietary_restrictions}."
    ),
    expected_output=f"One recipe using {main_ingredient} and matching {dietary_restrictions} restrictions.",
    agent=culinary_assistant,
)

# Task 2: Generate step-by-step cooking instructions
# 'context' links this task to the output of Task 1 — this is the stateful handoff
guide_recipe_steps = Task(
    description="Provide step-by-step instructions for the selected recipe.",
    expected_output="Step-by-step cooking instructions for the chosen recipe.",
    agent=culinary_assistant,
    context=[find_and_filter_recipes],  # <-- Receives Task 1's output automatically
)

## Step 5: Assemble the Crew & Execute

The **Crew** brings agents and tasks together. Setting `planning=True` enables CrewAI's built-in planner, which generates an execution plan before running the tasks.

When `kickoff()` is called, the crew:
1. Plans the execution order
2. Runs `find_and_filter_recipes` to find a suitable recipe
3. Passes that result into `guide_recipe_steps` via the `context` link
4. Returns the final cooking instructions

In [ ]:
crew = Crew(
    agents=[culinary_assistant],
    tasks=[find_and_filter_recipes, guide_recipe_steps],
    planning=True,
    planning_llm=llm,  # Use our Groq LLM for planning too (defaults to OpenAI otherwise)
)

result = crew.kickoff()
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Culinary Assistant                                                                                      │
│                                                                                                                 │
│  Task: Find recipes that use the ingredient: tomato and filter them to meet dietary restrictions: shrimps.1.    │
│  Open a web browser and navigate to a reputable recipe aggregation website such as Allrecipes, Food Network,    │
│  or Epicurious.                                                                                                 │
│  2. In the search bar, enter the query "tomato shrimp recipe" to retrieve recipes that explicitly list both     │
│  tomato and shrimp as primary ingredients.                                                                      │
│  3. Review the search results and click on each recipe title to open the full recipe page.                      │
│  4. On each recipe page, locate the ingredients list and confirm that tomato is listed as an ingredient.        │
│  5. Verify that shrimp is listed as the main protein source; if the recipe uses shrimp as a secondary           │
│  ingredient or garnish, note this but still consider it acceptable if the user’s dietary restriction is to      │
│  include shrimp.                                                                                                │
│  6. Check for any additional dietary restrictions that may conflict with the user’s preferences (e.g., gluten,  │
│  dairy, nuts). If the recipe contains conflicting ingredients, discard it.                                      │
│  7. Compile a shortlist of at least three recipes that meet the tomato and shrimp criteria and have no          │
│  conflicting dietary restrictions.                                                                              │
│  8. Evaluate each shortlisted recipe for overall quality: clear instructions, reasonable ingredient list, and   │
│  positive user reviews or ratings.                                                                              │
│  9. Select the single best recipe from the shortlist based on clarity, popularity, and suitability for the      │
│  user’s cooking skill level.                                                                                    │
│  10. Record the chosen recipe’s title, full ingredient list, and complete set of instructions for handover to   │
│  Task 2.                                                                                                        │
│  11. Prepare a concise summary of the selected recipe to provide to the user, including the recipe name,        │
│  servings, and a brief overview of the dish.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Culinary Assistant                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Tomato & Shrimp Pasta (Gluten‑Free, Dairy‑Free, Nut‑Free)**                                                  │
│  *Servings:* 4 | *Prep time:* 10 min | *Cook time:* 15 min                                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Ingredients                                                                                                │
│                                                                                                                 │
│  - 12 oz gluten‑free spaghetti (or your favorite gluten‑free pasta)                                             │
│  - 2 Tbsp olive oil                                                                                             │
│  - 4 cloves garlic, minced                                                                                      │
│  - 1/4 tsp red‑pepper flakes (optional)                                                                         │
│  - 1 (28‑oz) can whole‑tomato sauce (no added sugar or preservatives)                                           │
│  - 1 Tbsp tomato paste                                                                                          │
│  - 1 Tbsp fresh basil, chopped (or 1 tsp dried basil)                                                           │
│  - 1 Tbsp fresh oregano, chopped (or 1 tsp dried oregano)                                                       │
│  - 1 Tbsp fresh parsley, chopped                                                                                │
│  - 1 Tbsp lemon juice                                                                                           │
│  - 1 lb large shrimp, peeled & deveined                                                                         │
│  - Salt & freshly ground black pepper, to taste                                                                 │
│  - 1/4 cup grated Parmesan cheese (optional, omit for dairy‑free)                                               │
│  - Fresh basil leaves, for garnish                                                                              │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Instructions                                                                                               │
│                                                                                                                 │
│  1. **Cook the pasta**                                                                                          │
│     - Bring a large pot of salted water to a boil.                                                              │
│     - Add the gluten‑free spaghetti and cook according to package directions until al dente.                    │
│     - Reserve 1/2 cup of pasta water, then drain the pa

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Culinary Assistant                                                                                      │
│                                                                                                                 │
│  Task: Provide step-by-step instructions for the selected recipe.1. Retrieve the full recipe details obtained   │
│  from Task 1, ensuring you have the complete ingredient list and cooking instructions.                          │
│  2. Break down the recipe into a logical sequence of steps, numbering each step for clarity.                    │
│  3. For each step, include:                                                                                     │
│     - A concise action verb (e.g., "Chop", "Sauté", "Simmer").                                                  │
│     - The specific ingredients involved in that step.                                                           │
│     - The cooking method and any relevant temperature or time details.                                          │
│     - Any safety or technique tips (e.g., "Use a sharp knife to avoid slips", "Keep the heat medium to prevent  │
│  burning").                                                                                                     │
│  4. Add a preliminary section that lists:                                                                       │
│     - Total prep time and cook time.                                                                            │
│     - Number of servings.                                                                                       │
│     - Equipment needed (e.g., skillet, saucepan, cutting board).                                                │
│  5. Include a final plating or serving suggestion, such as garnish options or side dish recommendations.        │
│  6. Format the instructions in a user-friendly manner, using bullet points or numbered lists, and ensure the    │
│  language is clear and accessible to cooks of all skill levels.                                                 │
│  7. Review the entire instruction set for completeness, ensuring no step is omitted and that the flow from      │
│  prep to final dish is logical.                                                                                 │
│  8. Deliver the step-by-step guide to the user, ready for immediate use in the kitchen.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Culinary Assistant                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Tomato & Shrimp Pasta (Gluten‑Free, Dairy‑Free, Nut‑Free)**                                                  │
│  *Servings:* 4 | *Prep time:* 10 min | *Cook time:* 15 min                                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Equipment Needed                                                                                           │
│  - Large pot (for pasta)                                                                                        │
│  - Colander (for draining)                                                                                      │
│  - Large skillet or sauté pan (at least 12 in. wide)                                                            │
│  - Cutting board                                                                                                │
│  - Sharp knife (for chopping herbs)                                                                             │
│  - Measuring spoons & cups                                                                                      │
│  - Wooden spoon or spatula                                                                                      │
│  - Small bowl (for mixing shrimp seasoning)                                                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Step‑by‑Step Instructions                                                                                   │
│                                                                                                                 │
│  1. **Boil the Pasta**                                                                                          │
│     - **Action:** *Cook*                                                                                        │
│     - **Ingredients:** 12 oz gluten‑free spaghetti, salted water.                                               │
│     - **Method:** Fill the pot with water, add a generous pinch of salt, bring to a rolling boil. Add pasta,    │
│  stir once, and cook until al dente (usually 8–10 min).                                                         │
│     - **Tip:** Keep the heat medium‑high to maintain a steady boil; stir occasionally to prevent sticking.      │
│                                                                                                                 │
│  2. **Reserve Pasta Water**                                                                                     │
│     - **Action:** *Reserve*                                                                                     │
│     - **Ingredients:** 1/2 cup of pasta cooking water.                                                          │
│     - **Method:** Before draining, ladle out 1/2 cup of

**Tomato & Shrimp Pasta (Gluten‑Free, Dairy‑Free, Nut‑Free)**  
*Servings:* 4 | *Prep time:* 10 min | *Cook time:* 15 min  

---

### Equipment Needed  
- Large pot (for pasta)  
- Colander (for draining)  
- Large skillet or sauté pan (at least 12 in. wide)  
- Cutting board  
- Sharp knife (for chopping herbs)  
- Measuring spoons & cups  
- Wooden spoon or spatula  
- Small bowl (for mixing shrimp seasoning)  

---

## Step‑by‑Step Instructions  

1. **Boil the Pasta**  
   - **Action:** *Cook*  
   - **Ingredients:** 12 oz gluten‑free spaghetti, salted water.  
   - **Method:** Fill the pot with water, add a generous pinch of salt, bring to a rolling boil. Add pasta, stir once, and cook until al dente (usually 8–10 min).  
   - **Tip:** Keep the heat medium‑high to maintain a steady boil; stir occasionally to prevent sticking.  

2. **Reserve Pasta Water**  
   - **Action:** *Reserve*  
   - **Ingredients:** 1/2 cup of pasta cooking water.  
   - **Method:** Before draining, ladle 

In [19]:
print(result)

**Tomato & Shrimp Pasta (Gluten‑Free, Dairy‑Free, Nut‑Free)**  
*Servings:* 4 | *Prep time:* 10 min | *Cook time:* 15 min  

---

### Equipment Needed  
- Large pot (for pasta)  
- Colander (for draining)  
- Large skillet or sauté pan (at least 12 in. wide)  
- Cutting board  
- Sharp knife (for chopping herbs)  
- Measuring spoons & cups  
- Wooden spoon or spatula  
- Small bowl (for mixing shrimp seasoning)  

---

## Step‑by‑Step Instructions  

1. **Boil the Pasta**  
   - **Action:** *Cook*  
   - **Ingredients:** 12 oz gluten‑free spaghetti, salted water.  
   - **Method:** Fill the pot with water, add a generous pinch of salt, bring to a rolling boil. Add pasta, stir once, and cook until al dente (usually 8–10 min).  
   - **Tip:** Keep the heat medium‑high to maintain a steady boil; stir occasionally to prevent sticking.  

2. **Reserve Pasta Water**  
   - **Action:** *Reserve*  
   - **Ingredients:** 1/2 cup of pasta cooking water.  
   - **Method:** Before draining, ladle 